In [24]:
import pandas as pd
import numpy as np

df = pd.read_csv("./data/data_watermelon.csv")
df.head()

,序号,色泽,根蒂,敲声,纹理,脐部,触感,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,1
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,1
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,1
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,1
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,1


In [25]:
# 定义熵的计算
def calc_entropy(data):
    label = data.iloc[:, -1]
    label_count = label.value_counts()
    label_count = label_count / label_count.sum()
    entropy = (-label_count * np.log2(label_count)).sum()
    return entropy

# 定义信息增益的计算方法
def calc_info_gain(data, feature):
    entropy = calc_entropy(data)
    feature_count = data[feature].value_counts()
    feature_count = feature_count / feature_count.sum()
    feature_entropy = 0
    for i in feature_count.index:
        feature_entropy += calc_entropy(data[data[feature] == i]) * feature_count[i]
    info_gain = entropy - feature_entropy
    return info_gain

# 定义C4.5算法中信息增益比的计算。
def calc_info_gain_ratio(data, feature):
    info_gain = calc_info_gain(data, feature)
    feature_count = data[feature].value_counts()
    feature_entropy = -sum((feature_count / feature_count.sum()) * np.log2(feature_count / feature_count.sum()))
    if feature_entropy == 0:
        return 0
    return info_gain / feature_entropy

# 定义选择最佳特征的函数
def choose_best_feature(data, method):
    feature_list = data.columns[:-1]
    if method == "ID3":
        info_gain_list = [calc_info_gain(data, i) for i in feature_list]
    else:  # C4.5
        info_gain_list = [calc_info_gain_ratio(data, i) for i in feature_list]
    best_feature = feature_list[np.argmax(info_gain_list)]
    return best_feature

# 定义创建决策树的函数
def create_tree(data, method):
    label = data.iloc[:, -1]
    if label.nunique() == 1:
        return label.iloc[0]
    if len(data.columns) == 1:
        return label.value_counts().index[0]
    best_feature = choose_best_feature(data, method)
    tree = {best_feature: {}}
    for i in data[best_feature].unique():
        subtree = data[data[best_feature] == i].drop(best_feature, axis=1)
        tree[best_feature][i] = create_tree(subtree, method)
    return tree

# 定义修改后的预测函数
def predict_modified(tree, data, default_label):
    if not isinstance(tree, dict):
        return [tree] * len(data)

    feature = list(tree.keys())[0]
    subtree = tree[feature]
    label = data[feature]
    result = []
    for value in label:
        if value in subtree:
            result.append(predict_modified(subtree[value], data.drop(feature, axis=1), default_label)[0])
        else:
            result.append(default_label)
    return result

# 定义计算准确率的函数。
def calc_accuracy_modified(tree, data, default_label):
    label = data.iloc[:, -1]
    prediction = predict_modified(tree, data, default_label)
    accuracy = (np.array(prediction) == label).sum() / len(label)
    return accuracy

In [30]:
# 划分训练集和测试集 
df_train = df.sample(12)
df_test = df.drop(df_train.index)
default_label = df_train.iloc[:, -1].value_counts().idxmax()

In [31]:
# 训练和测试ID3决策树
tree_id3 = create_tree(df_train, "ID3")
accuracy_id3 = calc_accuracy_modified(tree_id3, df_test, default_label)
print("ID3 Decision Tree:\n", tree_id3)
print("ID3 Decision Tree Accuracy:", accuracy_id3)

# 训练和测试C4.5决策树
tree_c45 = create_tree(df_train, "C4.5")
accuracy_c45 = calc_accuracy_modified(tree_c45, df_test, default_label)
print("\nC4.5 Decision Tree:\n", tree_c45)
print("C4.5 Decision Tree Accuracy:", accuracy_c45)

ID3 Decision Tree:
 {'序号': {6: 1, 8: 1, 12: 0, 1: 1, 4: 1, 9: 0, 7: 1, 5: 1, 13: 0, 16: 0, 11: 0, 14: 0}}
ID3 Decision Tree Accuracy: 0.4

C4.5 Decision Tree:
 {'纹理': {'清晰': 1, '模糊': 0, '稍糊': {'触感': {'硬滑': 0, '软粘': 1}}}}
C4.5 Decision Tree Accuracy: 0.6


分别利用ID3和C4.5决策树模型，根据实验数据进行训练；
计算准确率，对比两种方法的结果。

In [26]:
def prune_tree(tree, data, threshold):
    # 预剪枝
    # tree: 决策树
    # data: 数据集
    # threshold: 剪枝阈值

    if len(data) < threshold:
        return tree

    best_feature = choose_best_feature(data, "C4.5")
    tree = {best_feature: {}}
    for i in data[best_feature].unique():
        tree[best_feature][i] = prune_tree(data[data[best_feature] == i].drop(best_feature, axis=1), data, threshold)
    return tree

In [27]:
def post_prune_tree(tree, data, X_val, y_val, threshold):
    # 后剪枝
    # tree: 决策树
    # data: 训练数据集
    # X_val: 验证数据集
    # y_val: 验证标签
    # threshold: 剪枝阈值

    best_feature = choose_best_feature(data, "C4.5")
    tree = {best_feature: {}}
    for i in data[best_feature].unique():
        tree[best_feature][i] = post_prune_tree(data[data[best_feature] == i].drop(best_feature, axis=1), X_val, y_val, threshold)

    # 计算交叉验证误差
    error = 0
    for i in range(len(X_val)):
        predict = predict(tree, X_val.iloc[i, :])
    if predict != y_val[i]:
        error += 1

    # 剪枝
    if error < threshold:
        return tree

    # 剪枝失败
    return None